In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
## Importando banco de dados covid_19_clean_complete.csv
df = pd.read_csv('/kaggle/input/corona-virus-report/covid_19_clean_complete.csv')
df.head(10)

**Parte 1**

In [4]:
## Quantidade de linhas e colunas
rows, columns = df.shape

## Lista de Colunas
columns_name = list(df.columns)

## Tipo de cada coluna
comluns_dtype = df.dtypes

## Transformando coluna de datas
df['Date'] = pd.to_datetime(df.Date)

## Colunas com NaN
df.columns[df.isna().any()].tolist()

## Informações Estatísticas
df.info()


**Parte 2**

In [5]:
## Provincias Chinesas
chinese_provinces = df[df['Country/Region'] == 'China']

## Dados Requeridos
chinese_provinces = chinese_provinces.loc[:,['Province/State','Confirmed', 'Deaths', 'Recovered', 'Active']]
provinces_info = chinese_provinces.groupby('Province/State').sum().sort_values('Confirmed', ascending=False)

## 5 provincias com mais casos
provinces_info.head(5)

**Parte 3**

In [6]:
## Função para unir países e provincias
def concatProvince(row):
    if pd.notna(row['Province/State']):
        row['Country/Region'] =  row['Country/Region'] + '-' + row['Province/State']
    return row

## Cópia do banco de dados
copy_df = df.copy()

## Usando o método apply
copy_df = copy_df.apply(concatProvince, axis=1)

## Excluindo coluna "Province/State"
del(copy_df['Province/State'])

copy_df.head(10)

**Parte 4**

In [17]:
## Importando banco de dados "world meter data.csv"
wmd = pd.read_csv('/kaggle/input/corona-virus-report/worldometer_data.csv')
wmd.head(10)

## Filtrando dados
df_infos = df[['Country/Region','Deaths']].groupby('Country/Region').sum()

## Mesclando "world meter data.csv" e “covid_19_clean_complete”
df_mix = df_infos.merge(wmd[['Country/Region','Population','Continent']], on = 'Country/Region')

## Agrupando DataFrame mesclado por continente
deaths_per_continent = df_mix.groupby('Continent', as_index=False).sum()

## Criando coluna "Deaths_per_Million"
deaths_per_continent['Deaths_per_Million'] = (deaths_per_continent['Deaths']/deaths_per_continent['Population'])*(10**6)

## Ranking
ranking = deaths_per_continent.sort_values(by = 'Deaths_per_Million', ascending=False)
ranking['Ranking'] = [i for i in range(1,len(deaths_per_continent)+1)]
ranking = ranking.set_index('Ranking')
ranking